# 4. Comparing Canonical Models

Authors: **Gorka Zamora-López** and **Matthieu Gilson**

---------------------

This notebook is part of an introductory tutorial for the use of *Stimulus-Response Network Analysis (SiReNetA)* to study the structure of complex networks:
1. *[Getting Started and Overview](1_GettingStarted.ipynb)*
2. *[Response to Stimulus](2_Basics_StimResp.ipynb)*
3. *[Extracting Metrics](3_Basics_ExtractMetrics.ipynb)*
4. **[Canonical Models](4_Basics_CanonMods.ipynb)**
5. *[Comparing Networks](5_UseCase_CompareNets.ipynb)*
6. *[Network Distance](6_UseCase_NetDist.ipynb)*
7. *[Weighted Networks](7_UseCase_WeighteNets.ipynb)* 

---------------------

### Outline and functions used in this notebook
* Compare the responses for different propagation models with the same binary network.
* Reproduce Fig 2 of [Zamora-López and Gilson (2024) Chaos](https://doi.org/10.1063/5.0202241)
* `Resp_DiscreteCascade` = `Resp_DC`
* `Resp_RandomWalk` = `Resp_RW`
* `Resp_ContCascade` = `Resp_CC`
* `Resp_LeakyCascade` = `Resp_LC`
* `Resp_ContDiffusion` = `Resp_CD`

---------------------

#### Import libraries

To get started, the first thing we need is to load the libraries we will need to work.

In [ ]:
# Python standard library imports
# Third party imports
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# import 
import sireneta as sna

## 0. Mathematical description of the propagation models

In all that follows, $A$ is the connectivity matrix, which is undirected binary graph here although this can be relaxed (see Notebook 7).

### Discrete Cascade

$ R^t = A^t $

### Random Walk

$ R^t = A^t $ with the Laplacian $L = A - D$ with $D$ the diagonal matrix with the corresponding degrees of nodes in $A$.

### Continuous Cascade

$ R(t) = e^{A t} $ defined using the matrix exponential.

### Continuous Leaky Cascade

$ R(t) = e^{J t} - e^{J^0 t} $ with $J^0 = \frac{1}{\tau} I$ being a diagonal matrix and the Jacobian $J = J^0 + A$. The role of the leakage time-constant $\tau$ is to control the rate of leakage through the nodes, thus counter-balancing the tendency of the linear connectivity to cause the divergence of the system.

### Continuous Diffusion

$ R(t) = e^{L t} $ with the same Laplacian as above.

    

## 1. Comparison of network responses for the 5 propagation models above for an example network

We will
* load an example network
* compute the network response for the propagation models
* compare their temporal evolution, in terms of global response and node responses

In [ ]:
# Load a small network (binary or unweighted)
net = np.loadtxt('../Data/Testnet_N8.txt', dtype=int)
N = len(net)

print(net)

In [ ]:
# normalize connectivity with respect to largest eigenvalue
lambda_max = np.real(np.linalg.eigvals(net)).max()
net_norm = net / lambda_max

# time specifics
dt = 0.1
T = 20.1
td = np.arange(0,T, dtype=int)
tc = np.arange(0,T,dt)
nT = td.size

rdc = sna.Resp_DiscreteCascade(net, tmax=20)
rrw = sna.Resp_RandomWalk(net, tmax=20)
rcc = sna.Resp_ContCascade(net, tmax=20, timestep=dt)
rlc = sna.Resp_LeakyCascade(net_norm, tau=0.8, tmax=20, timestep=dt)
rcd = sna.Resp_ContDiffusion(net, tmax=20, timestep=dt, case='full')

# list of responses
ts = [td, td, tc, tc, tc]
rs = [rdc, rrw, rcc, rlc, rcd]
labels = ['dc', 'rw', 'cc', 'lc', 'cd']
n_pm = 5

# match discrete continuous times
tdt = []
for t in td:
    # equivalent continuous time
    tdt.append(int(t/dt))

In [ ]:
plt.figure()
plt.subplot(211)
for i in range(0,3):
    plt.plot(ts[i], sna.GlobalResponse(rs[i]), label=labels[i])
plt.axis(ymin=0, ymax=100)
plt.legend(fontsize=10)
plt.subplot(212)
for i in range(3,5):
    plt.plot(ts[i], sna.GlobalResponse(rs[i]), label=labels[i])
plt.legend(fontsize=10)
plt.show()

In [ ]:
# calculate the node responses
inrs = []
onrs = []
for i in range(n_pm):
    inrs.append(sna.NodeResponses(rs[i])[0])
    onrs.append(sna.NodeResponses(rs[i])[1])

aspects = [0.5, 0.5, 5.0, 5.0, 5.0]

In [ ]:
# plot the node responses as curves

plt.figure(figsize=[10,10])
for i in range(n_pm):
    plt.subplot(n_pm,1,i+1)
    plt.plot(ts[i], inrs[i])
    plt.title(labels[i])
plt.tight_layout()
plt.show()

In [ ]:
#8 plot the node responses as carpet plots

plt.figure(figsize=[10,10])
for i in range(n_pm):
    plt.subplot(n_pm,1,i+1)
    plt.imshow(inrs[i].T, aspect=aspects[i], interpolation='nearest')
    plt.title(labels[i])
    plt.colorbar()
plt.tight_layout()
plt.show()